# Automated Essay Scoring 

### What is this Notebook about?

- This Notebook contains all raw results and experiments done with the various deep learning methods. The output cells contains the results that we got. The whole Notebook may take approximately 5-6 hours. This Notebook contains the following:

* Preprocess data
* Important util methods to be used later
* Deep Learning Model declaration
* Running bert for individual sets
* Running bert for whole dataset
* Running word2vec for individual sets
* Running word2vec for whole dataset

In [1]:
import tensorflow as tf
print(tf.__version__)
print(tf.config.list_physical_devices())

2.8.0
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [27]:
import sys
# !{sys.executable} -m pip install transformers

In [2]:
# import important libraries and download data
import os
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import cohen_kappa_score
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import multiprocessing
%matplotlib notebook
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')  
# ! git clone https://github.com/Gaurav-Pande/AES_DL.git && mv AES_DL/data .
# ! pip install transformers

ModuleNotFoundError: No module named 'gensim'

In [ ]:
# Declaring some visualization methods to plot accuracy and model diagram
def plot_accuracy_curve(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['mae'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

def plot_acrchitecture(filename, model):
    plot_model(model, to_file=str(filename) + '.png')

In [3]:
# method to split data into sets
def split_in_sets(data):
    essay_sets = []
    min_scores = []
    max_scores = []
    
    for s in range(1,9):
        essay_set = data[data["essay_set"] == s]
        essay_set.dropna(axis=1, inplace=True)
        n, d = essay_set.shape
        set_scores = essay_set["domain1_score"]
        print ("Set", s, ": Essays = ", n , "\t Attributes = ", d)
        min_scores.append(set_scores.min())
        max_scores.append(set_scores.max())
        essay_sets.append(essay_set)
    
    return (essay_sets, min_scores, max_scores)

In the below cell, we can see the data we need to operate. We essentially drops the column, we dont need and keep the domain_score only along with essay text.

In [4]:
dataset_path = "./data/training_set_rel3.tsv"
data = pd.read_csv(dataset_path, sep="\t", encoding="ISO-8859-1")
data.head()

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,...,rater2_trait3,rater2_trait4,rater2_trait5,rater2_trait6,rater3_trait1,rater3_trait2,rater3_trait3,rater3_trait4,rater3_trait5,rater3_trait6
0,1,1,"Dear local newspaper, I think effects computer...",4,4,NaN,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",5,4,NaN,9,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",4,3,NaN,7,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",5,5,NaN,10,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1,"Dear @LOCATION1, I know having computers has a...",4,4,NaN,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
data.shape

(12976, 28)

In [6]:
min_scores = [2, 1, 0, 0, 0, 0, 0, 0]
max_scores = [12, 6, 3, 3, 4, 4, 30, 60]
essay_sets, data_min_scores, data_max_scores = split_in_sets(data)
set1, set2, set3, set4, set5, set6, set7, set8 = tuple(essay_sets)
data.dropna(axis=1, inplace=True)
data.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
set1.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
set2.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
set3.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
set4.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
set5.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
set6.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
set7.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
set8.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
sets = [set1,set2,set3,set4,set5,set6,set7,set8]
data.head()

Set 1 : Essays =  1783 	 Attributes =  6
Set 2 : Essays =  1800 	 Attributes =  9
Set 3 : Essays =  1726 	 Attributes =  6
Set 4 : Essays =  1770 	 Attributes =  6
Set 5 : Essays =  1805 	 Attributes =  6
Set 6 : Essays =  1800 	 Attributes =  6
Set 7 : Essays =  1569 	 Attributes =  14
Set 8 : Essays =  723 	 Attributes =  18


/var/folders/pj/ldkmdmbj5bld43hhctfjwh280000gn/T/ipykernel_6368/2675578851.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  essay_set.dropna(axis=1, inplace=True)
/var/folders/pj/ldkmdmbj5bld43hhctfjwh280000gn/T/ipykernel_6368/1243294380.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  set1.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
/var/folders/pj/ldkmdmbj5bld43hhctfjwh280000gn/T/ipykernel_6368/1243294380.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

,essay_id,essay_set,essay,domain1_score
0,1,1,"Dear local newspaper, I think effects computer...",8
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10
4,5,1,"Dear @LOCATION1, I know having computers has a...",8


In [7]:
cap = ['@CAPS'+str(i) for i in range(100)]
loc = ['@LOCATION'+str(i) for i in range(100)]
org =['@ORGANIZATION'+str(i) for i in range(100)]
per = ['@PERSON'+str(i) for i in range(100)]
date = ['@DATE'+str(i) for i in range(100)]
time = ['@TIME'+str(i) for i in range(100)]
money = ['@MONEY'+str(i) for i in range(100)]
ner =  cap + loc + org + per + date + time + money

## Remove outliers

Removing essays have less than 100 words

In [35]:
essays_less_100_words = data[data.essay.str.len() < 100]
essays_less_100_words

,essay_id,essay_set,essay,domain1_score
356,359,1,"Being active has no limit, but technology does.",2
542,545,1,I think that computers are amazing. Computers ...,2
1781,1786,1,"Dear readers, I think that its good and bad to...",2
3648,6044,3,The setting affected the cyclist because of th...,2
3792,6188,3,The features of the setting do affect the cycl...,0
...,...,...,...,...
11144,18348,7,My dad can be very patient sometimes. When he...,5
11699,18958,7,I show loyalty a lot of peolpe that I no and t...,2
11748,19010,7,I do not have patience.,3
12150,19450,7,I was patient when I was at the @CAPS1.R whe I...,4


In [36]:
data = data[~data.isin(essays_less_100_words)].dropna()
data.head()

,essay_id,essay_set,essay,domain1_score
0,1.0,1.0,"Dear local newspaper, I think effects computer...",8.0
1,2.0,1.0,"Dear @CAPS1 @CAPS2, I believe that using compu...",9.0
2,3.0,1.0,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7.0
3,4.0,1.0,"Dear Local Newspaper, @CAPS1 I have found that...",10.0
4,5.0,1.0,"Dear @LOCATION1, I know having computers has a...",8.0


In [40]:
data.shape

(12889, 4)

## Normalize score

Each set has different score range, so we are going to normalize them to 0-1

In [8]:
topic_number = 0
fig, ax = plt.subplots(4,2, figsize=(7, 7), sharey=False)

for i in range(4):
    for j in range(2):
        topic_number += 1
        data[data['essay_set'] == topic_number].groupby('domain1_score')['essay_id'].agg('count').plot.bar(ax=ax[i, j], rot=0, color="maroon")
        ax[i,j].set_title('Essay %i' % topic_number)

ax[3,0].locator_params(nbins=10)
ax[3,1].locator_params(nbins=10)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

NameError: name 'plt' is not defined

In [9]:
for topic in range(1, 9):
    topic_set = data[data['essay_set'] == topic]
    min_set = topic_set['domain1_score'].min()
    max_set = topic_set['domain1_score'].max()

    data.loc[data['essay_set'] == topic, 'domain1_score'] = data[data['essay_set'] == topic]['domain1_score'].apply(lambda x: (x - min_set) / (max_set - min_set))
    # data[data['essay_set'] == topic]['domain1_score'].apply(lambda x: (x - min_set) / (max_set - min_set))
    # data[data['essay_set'] == topic]['domain1_score']

data

,essay_id,essay_set,essay,domain1_score
0,1,1,"Dear local newspaper, I think effects computer...",0.60
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",0.70
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",0.50
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",0.80
4,5,1,"Dear @LOCATION1, I know having computers has a...",0.60
...,...,...,...,...
12971,21626,8,In most stories mothers and daughters are eit...,0.50
12972,21628,8,I never understood the meaning laughter is th...,0.44
12973,21629,8,"When you laugh, is @CAPS5 out of habit, or is ...",0.60
12974,21630,8,Trippin' on fen...,0.60


In [10]:
topic_number = 0
fig, ax = plt.subplots(4,2, figsize=(7, 7), sharey=False)

for i in range(4):
    for j in range(2):
        topic_number += 1
        data[data['essay_set'] == topic_number].groupby('domain1_score')['essay_id'].agg('count').plot.bar(ax=ax[i, j], rot=0, color="maroon")
        ax[i,j].set_title('Essay %i' % topic_number)

ax[3,0].locator_params(nbins=10)
ax[3,1].locator_params(nbins=10)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

NameError: name 'plt' is not defined

# Model

In [11]:
from tensorflow.keras.layers import Embedding, Input, LSTM, Dense, Dropout, Lambda, Flatten, Bidirectional, Conv2D, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.models import Sequential,Model, load_model, model_from_config
import keras.backend as K
from tensorflow.keras.optimizers import Adam

In [12]:
def get_model(Hidden_dim1=400, Hidden_dim2=128, return_sequences = True, dropout=0.5, recurrent_dropout=0.4, input_size=768, activation='relu', bidirectional = False):
    """Define the model."""
    model = Sequential()
    if bidirectional:
        model.add(Bidirectional(LSTM(Hidden_dim1,return_sequences=return_sequences , dropout=0.4, recurrent_dropout=recurrent_dropout), input_shape=[1, input_size]))
        model.add(Bidirectional(LSTM(Hidden_dim2, recurrent_dropout=recurrent_dropout)))
    else:
        model.add(LSTM(Hidden_dim1, dropout=0.4, recurrent_dropout=recurrent_dropout, input_shape=[1, input_size], return_sequences=return_sequences))
        model.add(LSTM(Hidden_dim2, recurrent_dropout=recurrent_dropout))

    model.add(Dropout(dropout))
    model.add(Dense(64, activation=activation))
    model.add(Dense(32, activation=activation))
    model.add(Dense(1, activation=activation))

    model.compile(loss='mean_squared_error', optimizer='rmsprop',metrics=['mse'])
    model.summary()
    
    return model

## All sets using BERT

In [13]:
def get_features(X):
    max_len = 0
    for i in X.values:
        if len(i) > max_len:
            max_len = len(i)

    padded_X = np.array([i + [0]*(max_len-len(i)) for i in X.values])
    attention_mask_X = np.where(padded_X != 0, 1, 0)
    X_input_ids = torch.tensor(padded_X)
    X_attention_mask = torch.tensor(attention_mask_X)

    with torch.no_grad():
        last_hidden_states_X = model(X_input_ids, attention_mask=X_attention_mask)

    X_features = last_hidden_states_X[0][:,0,:].numpy()

    return X_features


In [78]:
## Sets experiment BERT
import time
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')
set_count = 1
all_sets_score = []

for s in sets:
    print("\n--------SET {}--------\n".format(set_count))
    X = s
    y = s['domain1_score']
    cv = KFold(n_splits=5, shuffle=True)
    cv_data = cv.split(X)
    results = []
    prediction_list = []
    fold_count =1
#     cuda = torch.device('cuda')
    model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
#     model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

    # Load pretrained model/tokenizer
    tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
    model = model_class.from_pretrained(pretrained_weights)

    for traincv, testcv in cv_data:
        print("\n--------Fold {}--------\n".format(fold_count))

        # get the train and test from the dataset.
        X_train, X_test, y_train, y_test = X.iloc[traincv], X.iloc[testcv], y.iloc[traincv], y.iloc[testcv]
        train_essays = X_train['essay']
        test_essays = X_test['essay']

        sentences = []
        tokenize_sentences = []
        train_bert_embeddings = []

        tokenized_train = train_essays.apply((lambda x: tokenizer.encode(x, add_special_tokens=True ,max_length=200)))
        tokenized_test = test_essays.apply((lambda x: tokenizer.encode(x, add_special_tokens=True ,max_length=200)))

        train_features = get_features(tokenized_train)
        test_features = get_features(tokenized_test)

        train_x, train_y = train_features.shape
        test_x, test_y = test_features.shape

        trainDataVectors = np.reshape(train_features,(train_x, 1, train_y))
        testDataVectors = np.reshape(test_features,(test_x, 1, test_y))

        # Bidirectional LSTM Model
        bi_dir_lstm_model = get_model(bidirectional=True)
        bi_dir_lstm_model.fit(trainDataVectors, y_train, batch_size=128, epochs=70)
        y_pred = bi_dir_lstm_model.predict(testDataVectors)
        y_pred = np.around(y_pred)
        #y_pred.dropna()
        np.nan_to_num(y_pred)

        # evaluate the model
        result = cohen_kappa_score(y_test.values, y_pred, weights='quadratic')
        print("Kappa Score: {}".format(result))
        results.append(result)
        fold_count +=1

        tf.keras.backend.clear_session()

        all_sets_score.append(results)

    print("Average kappa score value is : {}".format(np.mean(np.asarray(results))))
    set_count+=1
        # print(features.shape)


--------SET 1--------



Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the 


--------Fold 1--------

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 1, 800)           3740800   
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 256)              951296    
 nal)                                                            
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 64)                16448     
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (N

2022-04-16 23:04:34.480530: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


12/12 [==============================] - 7s 118ms/step - loss: 17.1625 - mse: 17.1625
Epoch 2/70
12/12 [==============================] - 1s 81ms/step - loss: 2.6606 - mse: 2.6606
Epoch 3/70
12/12 [==============================] - 1s 72ms/step - loss: 2.7873 - mse: 2.7873
Epoch 4/70
12/12 [==============================] - 1s 72ms/step - loss: 2.3790 - mse: 2.3790
Epoch 5/70
12/12 [==============================] - 1s 70ms/step - loss: 2.3948 - mse: 2.3948
Epoch 6/70
12/12 [==============================] - 1s 71ms/step - loss: 2.2457 - mse: 2.2457
Epoch 7/70
12/12 [==============================] - 1s 67ms/step - loss: 1.9842 - mse: 1.9842
Epoch 8/70
12/12 [==============================] - 1s 68ms/step - loss: 2.1276 - mse: 2.1276
Epoch 9/70
12/12 [==============================] - 1s 66ms/step - loss: 2.1237 - mse: 2.1237
Epoch 10/70
12/12 [==============================] - 1s 66ms/step - loss: 2.2012 - mse: 2.2012
Epoch 11/70
12/12 [==============================] - 1s 68ms/step -

...

Kappa Score: 0.5507206263221368

--------Fold 5--------

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 1, 800)           3740800   
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 256)              951296    
 nal)                                                            
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 64)                16448     
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 

2022-04-17 00:30:33.534554: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


5/5 [==============================] - 8s 352ms/step - loss: 1058.9618 - mse: 1058.9618
Epoch 2/70
5/5 [==============================] - 1s 195ms/step - loss: 174.7779 - mse: 174.7779
Epoch 3/70
5/5 [==============================] - 1s 138ms/step - loss: 43.1252 - mse: 43.1252
Epoch 4/70
5/5 [==============================] - 1s 123ms/step - loss: 38.6779 - mse: 38.6779
Epoch 5/70
5/5 [==============================] - 0s 94ms/step - loss: 38.0746 - mse: 38.0746
Epoch 6/70
5/5 [==============================] - 0s 100ms/step - loss: 38.4956 - mse: 38.4956
Epoch 7/70
5/5 [==============================] - 0s 87ms/step - loss: 37.7157 - mse: 37.7157
Epoch 8/70
5/5 [==============================] - 0s 82ms/step - loss: 35.7541 - mse: 35.7541
Epoch 9/70
5/5 [==============================] - 0s 83ms/step - loss: 34.7805 - mse: 34.7805
Epoch 10/70
5/5 [==============================] - 0s 78ms/step - loss: 35.5511 - mse: 35.5511
Epoch 11/70
5/5 [==============================] - 0s 84ms/

2022-04-17 00:31:06.407947: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Kappa Score: 0.5800317030025961
Average kappa score value is : 0.5602236187717858


In [23]:
data[:1]

,essay_id,essay_set,essay,domain1_score
0,1.0,1.0,"Dear local newspaper, I think effects computer...",0.6


In [25]:
import transformers as ppb
train_essays = data[:1]['essay']

model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

tokenized_train = train_essays.apply((lambda x: tokenizer.encode(x, add_special_tokens=True ,max_length=200)))
train_features = get_features(tokenized_train)
train_features

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the 

array([[ 5.23051694e-02,  1.60252471e-02,  2.04541191e-01,
         6.72682151e-02, -1.38523638e-01, -3.21713865e-01,
         2.45416909e-01,  7.12756455e-01, -3.32589209e-01,
        -3.69510531e-01,  4.78828922e-02, -1.46845967e-01,
         6.97909892e-02,  3.53184342e-01, -5.41524887e-02,
         1.71720386e-01,  2.62986263e-03,  3.73209089e-01,
         1.44100904e-01,  1.86410069e-01,  3.93429771e-02,
        -2.83513904e-01,  5.07952750e-01,  1.25221327e-01,
         2.49996811e-01, -1.34886950e-01,  1.15930252e-01,
         1.98495500e-02,  8.59399214e-02, -5.43922931e-02,
        -1.36439830e-01,  3.17826182e-01, -1.26200626e-02,
        -1.71214953e-01,  3.97089310e-02, -2.87185997e-01,
         9.30588320e-03, -8.81661028e-02,  2.33654715e-02,
         1.13749534e-01, -1.58142969e-01,  1.36231363e-01,
        -3.46561745e-02, -1.13332152e-01, -7.20796436e-02,
        -2.45297521e-01, -2.73655725e+00,  7.36569241e-02,
         8.46555829e-03, -1.23966180e-01,  1.31956488e-0

In [0]:
tf.keras.backend.clear_session()

In [0]:
trainDataVectors = np.reshape(train_features,(train_x,train_y,1))
testDataVectors = np.reshape(test_features,(test_x,test_y,1))
print(y_train.shape)
print(trainDataVectors.shape)

(10380,)
(10380, 768, 1)


In [1]:
# x = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
def plot_qwk_scores_all_sets():
  fig = plt.figure()
  ax = plt.subplot(111)
  x = [1,2,3,4,5]
  ax.plot(x, set1 , label='set1')
  ax.plot(x, set2, label='set2')
  ax.plot(x, set3, label='set3')
  ax.plot(x, set4, label='set4')
  ax.plot(x, set5, label='set5')
  ax.plot(x, set6, label='set6')
  ax.plot(x, set7, label='set7')
  ax.plot(x, set8, label='set8')
  plt.title('Set wise QWK using BERT for individual sets')
  ax.legend()
  plt.show()
